In [69]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

import pandas as pd

# initalise the tkinter GUI
root = tk.Tk()

root.geometry("500x500") # set the root dimensions
root.pack_propagate(False) # tells the root to not let the widgets inside it determine its size.
root.resizable(0, 0) # makes the root window fixed in size.

# Frame for TreeView
frame1 = tk.LabelFrame(root, text="Excel Data")
frame1.place(height=250, width=500)

# Frame for open file dialog
file_frame = tk.LabelFrame(root, text="Open File")
file_frame.place(height=200, width=500, rely=0.55, relx=0)
i = 0
# Buttons
button1 = tk.Button(file_frame, text="Browse", command=lambda: File_dialog())
button1.place(rely=0.85, relx=0)


button2 = tk.Button(file_frame, text="RestockReport", command=lambda: Load_excel_data())
button2.place(rely=0.85, relx=0.30)

button4 = tk.Button(file_frame, text="PrintListItems", command=lambda: File_dialog())
button4.place(rely=0.85, relx=0.60)

# The file/file path text


label_file = ttk.Label(file_frame, text="No File Selected")

label_file.place(rely=0, relx=0)

## Treeview Widget
tv1 = ttk.Treeview(frame1)
tv1.place(relheight=1, relwidth=1) # set the height and width of the widget to 100% of its container (frame1).

treescrolly = tk.Scrollbar(frame1, orient="vertical", command=tv1.yview) # command means update the yaxis view of the widget
treescrollx = tk.Scrollbar(frame1, orient="horizontal", command=tv1.xview) # command means update the xaxis view of the widget
tv1.configure(xscrollcommand=treescrollx.set, yscrollcommand=treescrolly.set) # assign the scrollbars to the Treeview Widget
treescrollx.pack(side="bottom", fill="x") # make the scrollbar fill the x axis of the Treeview widget
treescrolly.pack(side="right", fill="y") # make the scrollbar fill the y axis of the Treeview widget

filelist =[]
def File_dialog():
    """This Function will open the file explorer and assign the chosen file path to label_file"""
    filename = filedialog.askopenfilename(initialdir="/",
                                          title="Select A File",
                                          filetype=(("xlsx files", "*.xlsx"),("All Files", "*.*")))
    
    if label_file["text"] =="No File Selected":
        
        label_file["text"] = filename
        filelist.append(filename)
    else:
        name = label_file["text"] 
        
        label_file["text"] = name +" # "+ filename
        filelist.append(filename)
    print(filelist)    
    return None


def Load_excel_data():
    """If the file selected is valid this will load the file into the Treeview"""
    file_path = label_file["text"]
    
    try:
        excel_filename = r"{}".format(file_path)
        if excel_filename[-4:] == ".csv":
            df = pd.read_csv(excel_filename)
        else:
            df = pd.read_excel(excel_filename)

    except ValueError:
        tk.messagebox.showerror("Information", "The file you have chosen is invalid")
        return None
    except FileNotFoundError:
        tk.messagebox.showerror("Information", f"No such file as {file_path}")
        return None

    clear_data()
    tv1["column"] = list(df.columns)
    tv1["show"] = "headings"
    for column in tv1["columns"]:
        tv1.heading(column, text=column) # let the column heading = column name

    df_rows = df.to_numpy().tolist() # turns the dataframe into a list of lists
    for row in df_rows:
        tv1.insert("", "end", values=row) # inserts each list into the treeview. For parameters see https://docs.python.org/3/library/tkinter.ttk.html#tkinter.ttk.Treeview.insert
    return None


def clear_data():
    tv1.delete(*tv1.get_children())
    return None


root.mainloop()

[]
['C:/Users/user/Downloads/Task FBA/RestockReport.xlsx']
['C:/Users/user/Downloads/Task FBA/RestockReport.xlsx', 'C:/Users/user/Downloads/Task FBA/SalesReportFile.xlsx']


In [51]:
!pip install timedelta

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for timedelta: filename=timedelta-2020.12.3-py3-none-any.whl size=1603 sha256=94c2071846786aa25cf82b2556f2922cf2c95d10b1d439a571b1c192c5726631
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\0b\9a\39\614edf72f37e354917a992c40bd518b99fc9e47b73fffdee02
Successfully built timedelta


You should consider upgrading via the 'C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [96]:
import pandas as pd 
import datetime
da = datetime.datetime.now().date()

res = (pd.Timestamp(da) + pd.DateOffset(days=7)).strftime('%Y-%m-%d')


Rr = pd.read_excel("RestockReport.xlsx",parse_dates=['Recommended ship date'])
Pl = pd.read_excel("PrintListItems.xlsx")
df = pd.concat([Rr,Pl],axis="columns")
print(len(df))

res = res+" 00:00:00"
checklist =['LAN-RT-AP-BNC','FLG-N-CHE22', 'FLG-N-KH22', 'CH-N-NAS22']
#print(df['Recommended ship date'][2])
for i in range(0,len(df)):
    search =df['Merchant SKU'][i]
    #print(search in set(df['SKU']))
    
    
    if (search in set(df['SKU']) or df['Recommended ship date'][i] > res or df['Recommended replenishment qty'][i] <= 0):
            #df = df.drop([j], inplace = True)
        #print(search)
            #print(df['Merchant SKU'][j])
            #print("hi Iam here ")
           
        df.drop(i, inplace=True)
        """try:
            if df['SKU'][i] == df['Merchant SKU'][j]:
                df = df.drop([i], axis=0, inplace = True)
                df = df.drop([j], axis=0, inplace = True)
      except:
            print("none type")"""
    elif df['Recommended replenishment qty'][i] >0 and df['Total Units'][i] >=12 and df['Merchant SKU'][i] not in checklist:
         df.drop(i, inplace=True)
    
    elif df['Recommended replenishment qty'][i] < 3:
        df.iloc[i,26] = 3
        #df['Recommended replenishment qty'][i] = 3
        
print(len(df))



348
67


In [97]:
print(df.columns)

Index(['Country/Region Code', 'Product name', 'FNSKU', 'Merchant SKU', 'ASIN',
       'Condition', 'Supplier', 'Supplier part no.', 'Currency code', 'Price',
       'Sales last 30 days', 'Units Sold Last 30 Days', 'Total Units',
       'Inbound', 'Available', 'FC transfer', 'FC Processing',
       'Customer Order', 'Unfulfillable', 'Working', 'Shipped', 'Receiving',
       'Fulfilled by',
       'Total days of supply (including units from open shipments)',
       'Days of supply at Amazon fulfillment centers', 'Alert',
       'Recommended replenishment qty', 'Recommended ship date',
       'Recommended action', 'SKU', 'Item Quantity', 'Item Name',
       'Shipping method', 'Order date', 'Ship by date', 'Order Number',
       'Image URL', 'OrderItemId', 'Categorization Comments'],
      dtype='object')


In [98]:
df =df.drop(['Country/Region Code', 'Product name', 'FNSKU', 'Merchant SKU', 'ASIN',
       'Condition', 'Supplier', 'Supplier part no.', 'Currency code', 'Price',
       'Sales last 30 days', 'Units Sold Last 30 Days', 'Total Units',
       'Inbound', 'Available', 'FC transfer', 'FC Processing',
       'Customer Order', 'Unfulfillable', 'Working', 'Shipped', 'Receiving',
       'Fulfilled by',
       'Total days of supply (including units from open shipments)',
       'Days of supply at Amazon fulfillment centers', 'Alert',
       'Recommended replenishment qty', 'Order date',
       'Recommended action', 'SKU', 
       'Shipping method',  'Ship by date', 
       'Image URL', 'OrderItemId', 'Categorization Comments'], axis = 1)


In [99]:
print(df)

    Recommended ship date  Item Quantity  \
0     2022-06-09 00:00:00            6.0   
2     2022-05-27 00:00:00           12.0   
5     2022-05-27 00:00:00           24.0   
6     2022-05-27 00:00:00            6.0   
11    2022-05-27 00:00:00           24.0   
..                    ...            ...   
156   2022-05-27 00:00:00            NaN   
157   2022-06-02 00:00:00            NaN   
158   2022-06-07 00:00:00            NaN   
159   2022-06-05 00:00:00            NaN   
163   2022-05-27 00:00:00            NaN   

                                             Item Name Order Number  
0    Kentucky Wildcats New Mailbox Cover Design for...   FBA5242022  
2    Kyle Busch #18 Racing Nascar Deluxe Credential...   FBA5242022  
5                Officially Licensed NASCAR Can Hugger   FBA5242022  
6    Tampa Spartans Collegiate 3' x 5' Double Sided...   FBA5242022  
11   Chase Elliott #9 Nascar 3' x 5' Car Flag New f...   FBA5242022  
..                                                 

In [66]:

df.head()
df.to_excel("my.xlsx")

In [87]:
Sr = pd.read_excel("SalesReportFile.xlsx")
print(len(Sr))
for i in range(0,len(Sr)):
    name =Sr['fulfillment'][i] 
    
    
    if name != "Seller" or Sr['quantity'][i] < 0 or Sr['sku'][i] == " " or 'Custom' in Sr['description'][i] or 'Customizable' in Sr['description'][i] :
        Sr.drop(i, inplace=True)
    


6782


In [100]:
print(Sr.columns)


Index(['date/time', 'settlement id', 'type', 'order id', 'sku', 'description',
       'quantity', 'marketplace', 'account type', 'fulfillment', 'order city',
       'order state', 'order postal', 'tax collection model', 'product sales',
       'product sales tax', 'shipping credits', 'shipping credits tax',
       'gift wrap credits', 'giftwrap credits tax', 'Regulatory Fee',
       'Tax On Regulatory Fee', 'promotional rebates',
       'promotional rebates tax', 'marketplace withheld tax', 'selling fees',
       'fba fees', 'other transaction fees', 'other', 'total'],
      dtype='object')


In [101]:
Sr =Sr.drop([ 'settlement id', 'type', 'order id'
             , 'marketplace', 'account type', 'fulfillment', 'order city',
       'order state', 'order postal', 'tax collection model', 'product sales',
       'product sales tax', 'shipping credits', 'shipping credits tax',
       'gift wrap credits', 'giftwrap credits tax', 'Regulatory Fee',
       'Tax On Regulatory Fee', 'promotional rebates',
       'promotional rebates tax', 'marketplace withheld tax', 'selling fees',
       'fba fees', 'other transaction fees', 'other', 'total'], axis = 1)

In [102]:
print(Sr)

                        date/time                       sku  \
0     May 1, 2022 12:11:00 AM PDT       CST4-W-CMP-ELPA 482   
45    May 1, 2022 11:45:15 AM PDT  MGC4-CTY-S-OCAL Ret 1160   
46    May 1, 2022 11:45:16 AM PDT              ST5D-C-ORS14   
47    May 1, 2022 11:45:16 AM PDT              STSM-C-ORS14   
48    May 1, 2022 11:45:38 AM PDT                PNT-N-KL22   
...                           ...                       ...   
6700  May 17, 2022 9:17:43 AM PDT               LTS-C-NEB16   
6702  May 17, 2022 9:18:52 AM PDT  STC4-S-CMP-OAKH 3143 4pk   
6703  May 17, 2022 9:19:05 AM PDT         SGS-S-SMKYMNT18 B   
6704  May 17, 2022 9:19:05 AM PDT             SGR-GATTN18 A   
6770  May 17, 2022 9:53:45 PM PDT   SGS-CTY-S-DEST Ret 3290   

                                            description  quantity  
0     El Paso Texas 4 Pack Engraved Wooden Coaster C...       2.0  
45    Ocala Florida 5 x 2.5-Inch Fridge Magnet Retro...       1.0  
46    R and R Imports, Inc Oregon State

In [88]:
print(len(Sr))

4037


In [20]:
Rr.head()

,Country/Region Code,Product name,FNSKU,SKU,ASIN,Condition,Supplier,Supplier part no.,Currency code,Price,...,Working,Shipped,Receiving,Fulfilled by,Total days of supply (including units from open shipments),Days of supply at Amazon fulfillment centers,Alert,Recommended replenishment qty,Recommended ship date,Recommended action
0,US,RealTree Camo Pattern Hunting Keychain Breakaw...,X001VXW6Q7,LAN-RT-AP-BNC,B07BZ5MM95,New,unassigned,NaN,USD,7.99,...,0,0,0,Amazon,32,31,NaN,406,2022-06-09 00:00:00,Create shipping plan
1,US,"Officially Licensed NASCAR 4"" Round Kiss Cut D...",X002ODQAWT,STC4-N-NAS,B084JQHVXH,New,unassigned,NaN,USD,7.99,...,0,0,0,Amazon,13,13,low_stock,76,2022-05-27 00:00:00,Create shipping plan
2,US,R and R Imports Nascar Credential Holder with ...,B09SBNSKL9,CH-N-NAS22,B09SBNSKL9,New,unassigned,NaN,USD,11.99,...,0,0,0,Amazon,26,20,NaN,58,2022-05-27 00:00:00,Create shipping plan
3,US,Chase Elliott #9 Nascar 3' x 5' Car Flag New f...,B09SBRHVDD,FLG-N-CHE22,B09SBRHVDD,New,unassigned,NaN,USD,24.99,...,0,0,0,Amazon,31,31,NaN,56,2022-05-27 00:00:00,Create shipping plan
4,US,Kevin Harvick #4 Nascar 3' x 5' Car Flag New f...,B09SBT5KPT,FLG-N-KH22,B09SBT5KPT,New,unassigned,NaN,USD,24.99,...,0,0,0,Amazon,18,16,NaN,54,2022-05-27 00:00:00,Create shipping plan


In [21]:
df = pd.concat([Rr,Pl],axis="columns")
df

,Country/Region Code,Product name,FNSKU,Merchant SKU,ASIN,Condition,Supplier,Supplier part no.,Currency code,Price,...,SKU,Item Quantity,Item Name,Shipping method,Order date,Ship by date,Order Number,Image URL,OrderItemId,Categorization Comments
0,US,RealTree Camo Pattern Hunting Keychain Breakaw...,X001VXW6Q7,LAN-RT-AP-BNC,B07BZ5MM95,New,unassigned,NaN,USD,7.99,...,BXC-C-KEN21,6.0,Kentucky Wildcats New Mailbox Cover Design for...,UPS,NaN,2022-05-24,FBA5242022,NaN,NaN,This row is categorized to [Roland]
1,US,"Officially Licensed NASCAR 4"" Round Kiss Cut D...",X002ODQAWT,STC4-N-NAS,B084JQHVXH,New,unassigned,NaN,USD,7.99,...,BXC-C-KEN21,2.0,Kentucky Wildcats New Mailbox Cover Design for...,UPS,NaN,2022-05-24,FBA5242022,NaN,NaN,This row is categorized to [Roland]
2,US,R and R Imports Nascar Credential Holder with ...,B09SBNSKL9,CH-N-NAS22,B09SBNSKL9,New,unassigned,NaN,USD,11.99,...,CHD-N-KYB20,12.0,Kyle Busch #18 Racing Nascar Deluxe Credential...,UPS,NaN,2022-05-24,FBA5242022,NaN,NaN,This row is categorized to [Imports]
3,US,Chase Elliott #9 Nascar 3' x 5' Car Flag New f...,B09SBRHVDD,FLG-N-CHE22,B09SBRHVDD,New,unassigned,NaN,USD,24.99,...,CH-N-CHE22,12.0,R and R Imports Chase Elliott #9 Nascar Creden...,UPS,NaN,2022-05-24,FBA5242022,NaN,NaN,This row is categorized to [Imports]
4,US,Kevin Harvick #4 Nascar 3' x 5' Car Flag New f...,B09SBT5KPT,FLG-N-KH22,B09SBT5KPT,New,unassigned,NaN,USD,24.99,...,CH-N-KYB22,6.0,R and R Imports Kyle Busch #18 Nascar Credenti...,UPS,out_of_stock,2022-05-24,FBA5242022,NaN,NaN,This row is categorized to [Imports]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,US,Marshall Thundering Herd Rubber Trailer Hitch ...,X0030R3265,THR-C-MAR16,B01LZH2J8H,New,unassigned,NaN,USD,11.99,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
344,US,VMI Keydets Rubber Trailer Hitch Cover,X002O89VV1,THR-C-VMI16,B01LXV3EPP,New,unassigned,NaN,USD,11.99,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
345,US,West Virginia Mountaineers Rubber Trailer Hitc...,X002T432QF,THR-C-WVU16,B01M1441VM,New,unassigned,NaN,USD,11.99,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
346,US,Gonzaga Bulldogs Etched Stemless Wine Glass 9 ...,X002T432RT,WGE2-C-GNZ20,B08JKRR46R,New,unassigned,NaN,USD,24.99,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [7]:
import datetime
da = str(datetime.datetime.now().date())
da

'2022-06-02'

In [16]:
a = Rr['Recommended ship date'][1]
a[0:10]

'2022-05-27'

In [18]:
for i in range(0,len(Rr['Recommended ship date'])):
    a =Rr['Recommended ship date'][i]
    b = a[0:10]
    Rr['Recommended ship date'][i] = b 

C:\Users\user\AppData\Local\Temp/ipykernel_13776/1765955070.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Rr['Recommended ship date'][i] = b


In [6]:
df = Rr['']+Pl['']

KeyError: ''